# Digit MNIST Classifier

Torch version.

Found out that torch and numpy have similar syntax, so I try to create another version with torch and with some support to GPU compute (Kinda overkill lol)

Coded and Created by Han Summer 2024

Part of The 20th Summer Project

In [275]:
import torch
import torch.nn.functional as F
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [276]:
df=pd.read_csv('train.csv')

In [277]:
df.head(1).value_counts()

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  pixel10  pixel11  pixel12  pixel13  pixel14  pixel15  pixel16  pixel17  pixel18  pixel19  pixel20  pixel21  pixel22  pixel23  pixel24  pixel25  pixel26  pixel27  pixel28  pixel29  pixel30  pixel31  pixel32  pixel33  pixel34  pixel35  pixel36  pixel37  pixel38  pixel39  pixel40  pixel41  pixel42  pixel43  pixel44  pixel45  pixel46  pixel47  pixel48  pixel49  pixel50  pixel51  pixel52  pixel53  pixel54  pixel55  pixel56  pixel57  pixel58  pixel59  pixel60  pixel61  pixel62  pixel63  pixel64  pixel65  pixel66  pixel67  pixel68  pixel69  pixel70  pixel71  pixel72  pixel73  pixel74  pixel75  pixel76  pixel77  pixel78  pixel79  pixel80  pixel81  pixel82  pixel83  pixel84  pixel85  pixel86  pixel87  pixel88  pixel89  pixel90  pixel91  pixel92  pixel93  pixel94  pixel95  pixel96  pixel97  pixel98  pixel99  pixel100  pixel101  pixel102  pixel103  pixel104  pixel105  pixel106  pixel107  pixel108  pixel109  pix

In [278]:
df.shape

(42000, 785)

42k train set, with flattened 28x28 image => 784

Label -> 0-9

In [279]:
data=torch.tensor(df.values)
data.dtype

torch.int64

In [280]:
data=data.float()
data.dtype

torch.float32

In [281]:
data.shape

torch.Size([42000, 785])

In [282]:
#Move data to GPU
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data=data.to(device)

Shuffle

In [283]:
data.size() , data.size(0) , data.size(1)

(torch.Size([42000, 785]), 42000, 785)

In [284]:
torch.randperm(data.size(0)) #it creates a shuffled sequence of row indices.

tensor([ 7621, 35051, 19439,  ..., 38186, 38679, 20111])

In [285]:
#Shuffle tensor
data=data[torch.randperm(data.size(0))]

In [286]:
data_dev=data[:16800].T
data_dev.shape

torch.Size([785, 16800])

In [287]:
# Step 3: Find unique values
unique_values = torch.unique(data)

# Step 4: Print unique values
print(unique_values)

tensor([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,
         24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,
         36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,
         48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,
         60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
         72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,
         84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,
         96.,  97.,  98.,  99., 100., 101., 102., 103., 104., 105., 106., 107.,
        108., 109., 110., 111., 112., 113., 114., 115., 116., 117., 118., 119.,
        120., 121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
        132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
        144., 145., 146., 147., 148., 14

In [288]:
Y_dev=data_dev[0]
X_dev=data_dev[1:]
X_dev=X_dev/255.0 #normalize the data
X_dev.shape

torch.Size([784, 16800])

In [289]:
# Step 3: Find unique values
unique_values = torch.unique(X_dev)

# Step 4: Print unique values
print(unique_values)

tensor([0.0000, 0.0039, 0.0078, 0.0118, 0.0157, 0.0196, 0.0235, 0.0275, 0.0314,
        0.0353, 0.0392, 0.0431, 0.0471, 0.0510, 0.0549, 0.0588, 0.0627, 0.0667,
        0.0706, 0.0745, 0.0784, 0.0824, 0.0863, 0.0902, 0.0941, 0.0980, 0.1020,
        0.1059, 0.1098, 0.1137, 0.1176, 0.1216, 0.1255, 0.1294, 0.1333, 0.1373,
        0.1412, 0.1451, 0.1490, 0.1529, 0.1569, 0.1608, 0.1647, 0.1686, 0.1725,
        0.1765, 0.1804, 0.1843, 0.1882, 0.1922, 0.1961, 0.2000, 0.2039, 0.2078,
        0.2118, 0.2157, 0.2196, 0.2235, 0.2275, 0.2314, 0.2353, 0.2392, 0.2431,
        0.2471, 0.2510, 0.2549, 0.2588, 0.2627, 0.2667, 0.2706, 0.2745, 0.2784,
        0.2824, 0.2863, 0.2902, 0.2941, 0.2980, 0.3020, 0.3059, 0.3098, 0.3137,
        0.3176, 0.3216, 0.3255, 0.3294, 0.3333, 0.3373, 0.3412, 0.3451, 0.3490,
        0.3529, 0.3569, 0.3608, 0.3647, 0.3686, 0.3725, 0.3765, 0.3804, 0.3843,
        0.3882, 0.3922, 0.3961, 0.4000, 0.4039, 0.4078, 0.4118, 0.4157, 0.4196,
        0.4235, 0.4275, 0.4314, 0.4353, 

In [290]:
#Check if the number is normalized
X_dev.min() , X_dev.max()

(tensor(0., device='cuda:0'), tensor(1., device='cuda:0'))

In [291]:
data_train=data[16800:].T
Y_train=data_train[0]
X_train=data_train[1:]
X_train=X_train/255.0
X_train.shape

torch.Size([784, 25200])

In [292]:
X_train.dtype

torch.float32

In [293]:
X_train.device

device(type='cuda', index=0)

In [294]:
Y_train.dtype

torch.float32

In [295]:
# Step 3: Find unique values
unique_values = torch.unique(X_train)

# Step 4: Print unique values
print(unique_values)

tensor([0.0000, 0.0039, 0.0078, 0.0118, 0.0157, 0.0196, 0.0235, 0.0275, 0.0314,
        0.0353, 0.0392, 0.0431, 0.0471, 0.0510, 0.0549, 0.0588, 0.0627, 0.0667,
        0.0706, 0.0745, 0.0784, 0.0824, 0.0863, 0.0902, 0.0941, 0.0980, 0.1020,
        0.1059, 0.1098, 0.1137, 0.1176, 0.1216, 0.1255, 0.1294, 0.1333, 0.1373,
        0.1412, 0.1451, 0.1490, 0.1529, 0.1569, 0.1608, 0.1647, 0.1686, 0.1725,
        0.1765, 0.1804, 0.1843, 0.1882, 0.1922, 0.1961, 0.2000, 0.2039, 0.2078,
        0.2118, 0.2157, 0.2196, 0.2235, 0.2275, 0.2314, 0.2353, 0.2392, 0.2431,
        0.2471, 0.2510, 0.2549, 0.2588, 0.2627, 0.2667, 0.2706, 0.2745, 0.2784,
        0.2824, 0.2863, 0.2902, 0.2941, 0.2980, 0.3020, 0.3059, 0.3098, 0.3137,
        0.3176, 0.3216, 0.3255, 0.3294, 0.3333, 0.3373, 0.3412, 0.3451, 0.3490,
        0.3529, 0.3569, 0.3608, 0.3647, 0.3686, 0.3725, 0.3765, 0.3804, 0.3843,
        0.3882, 0.3922, 0.3961, 0.4000, 0.4039, 0.4078, 0.4118, 0.4157, 0.4196,
        0.4235, 0.4275, 0.4314, 0.4353, 

In [296]:
#Converting Y_train to int64 since class is not a float
# And F.one_hot only receive torch.int64 tensor
Y_train=Y_train.long()
Y_train.dtype

torch.int64

In [297]:
X_train.shape , Y_train.shape

(torch.Size([784, 25200]), torch.Size([25200]))

In [298]:
#Init params for 10 neurons and output layers
def initialize_parameters():
    #First layer -> input is 784 and there are 10 neurons
    W1= torch.rand((10,784)) -0.5
    b1= torch.rand((10,1))-0.5
    #Second layer -> input is 10 and output is 10 since there are 10 classes
    W2= torch.rand((10,10))-0.5
    b2= torch.rand((10,1)) -0.5
    #Move params to GPU
    W1=W1.to(device)
    b1=b1.to(device)
    W2=W2.to(device)
    b2=b2.to(device)
    return W1,b1,W2,b2

I tried the **torch.randn** function for init the params but when backward pass there is a problem which when summing the Z it would return NaN. 

AI told this is the problem caused by:
- Overflow or underflow: If the values in your tensor are extremely large or small, summing them can lead to numerical instability.
- Invalid operations: Operations like dividing by zero or taking the logarithm of a negative number can produce NaN values.
- Propagation of NaN: If your tensor already contains NaN values, any operation involving those values will result in NaN.
- Gradients exploding or vanishing: In the context of neural networks, this can happen during backpropagation.
- Infinities in the Tensor: If your tensor contains inf or -inf values, the sum operation could also return NaN. You can check for infinities using Z1.isinf().any().

After I checked that the NaN value or Inf are not present in the Tensor, 

and I found out that it is **Vanishing/Exploding Gradients** problem: this **randn** initialization can cause gradients to become very small (vanishing) or very large (exploding) as they propagate through the network.

how to fix it? Using Xavier/Glorot Init or He init(Good for ReLU)

In [299]:
#Forward pass
def ReLU(z):
    return torch.max(torch.tensor(0),z)
def Softmax(z):
    
    count= z.exp()
    # print(count.shape)
    prob= count/count.sum(0, keepdim=True)
    # print(count.sum(0, keepdim=True))
    return prob

def forward(X,W1,b1,W2,b2):
    # print( X.shape,W1.shape, b1.shape,W2.shape,b2.shape)
    Z1=W1@X+b1 #Formely W1.dot(X)+b1 in numpy
    A1=ReLU(Z1)
    Z2=W2@A1+b2 #Formely W2.dot(A1)+b2 in numpy
    A2=Softmax(Z2)
    return Z1,A1,Z2,A2
    

In [300]:
#Check softmax
z=torch.tensor([1.0,2.0,3.0])
Softmax(z).sum(0)

tensor(1.)

In [301]:
#Backward pass
def dReLU(z):
    # print((z>0).float())
    return (z>0).float()
# dReLU(torch.tensor([-1,0,1])) #Checking Relu

def backward(Z1,A1,Z2,A2,W2,X,Y):
    # print(Z1,A1,Z2,A2,W2,X,Y)
    m=Y_train.size(0)
    one_hot_Y= F.one_hot(Y_train.long(), num_classes=10).T #Transpose the matrix, because the matrix should be each column as a sample of data
    #--------------------
    dZ2= A2-one_hot_Y
    # print(dZ2)
    dW2= 1/m*dZ2@(A1.T)
    # print(dW2)
    db2= (1/m)*(dZ2.sum(1,keepdim=True)) #Dont forget the keep dim
    # print(db2,dZ2.sum(1,keepdim=True))
    # print(dZ2.shape, dZ2.sum(1,keepdim=True).shape)
    # print(db2.shape)
    #--------------------
    dZ1= (W2.T)@dZ2*dReLU(Z1)
    # print(dZ1)
    dW1= 1/m*dZ1@(X.T)
    # print(dW1)
    db1= (1/m)*(dZ1.sum(1,keepdim=True)) #Dont forget the keep dim
    # print(dW1.shape, db1.shape, dW2.shape, db2.shape)
    # print(db1,dZ1.sum(1,keepdim=True) )
    # print(dZ1.shape, dZ1.sum(1,keepdim=True).shape)
    # print(db1.shape)
    return dW1,db1,dW2,db2


In [302]:

def update (W1,b1,W2,b2,dW1,db1,dW2,db2,lr):
    W1=W1-lr*dW1
    # print(b1.shape, db1.shape)
    b1=b1-lr*db1
    W2=W2-lr*dW2
    # print(b2.shape, db2.shape)
    b2=b2-lr*db2
    # print(W1.shape, b1.shape, W2.shape, b2.shape)
    # print(dW1,dW2,db1,db2)
    # print(W1,b1,W2,b2)
    return W1,b1,W2,b2


In [303]:
#Training
def predict (A2):
    #Return the index of the maximum value in the array
    return torch.argmax(A2,0)

def accuracy(Y,Y_pred):
    return (Y==Y_pred).sum()/Y.size(0)

In [306]:
def gradient_descent(X,Y,alpha,epochs):
    W1,b1,W2,b2=initialize_parameters()
    for i in range(epochs):
        Z1,A1,Z2,A2=forward(X,W1,b1,W2,b2)
        dW1,db1,dW2,db2=backward(Z1,A1,Z2,A2,W2,X,Y)
        W1,b1,W2,b2=update(W1,b1,W2,b2,dW1,db1,dW2,db2,alpha)
        # print(Z1,A1,Z2,A2)
        # print(W1,b1,W2,b2)
        if i%10==0: #Print the loss every 10 epochs
            print('Iteration/Epoch: ',i)
            acctrain=accuracy(predict(A2),Y)
            print('Accuracy: ',acctrain.item())
            Z1,A1,Z2,A2=forward(X_dev,W1,b1,W2,b2)
            # print('Acc valll----------------------')
            accval=accuracy(predict(A2),Y_dev).item()
            print('Validation Accuracy: ',accval)
            # print(Z1,A1,Z2,A2)
            # print(W1,b1,W2,b2)
            # if (abs(acctrain-accval)>0.01):
            #     break
    return W1,b1,W2,b2

In [307]:
W1,b1,W2,b2= gradient_descent(X_train,Y_train,0.1,2500)

Iteration/Epoch:  0
Accuracy:  0.07305555045604706
Validation Accuracy:  0.07880952209234238
Iteration/Epoch:  10
Accuracy:  0.15813490748405457
Validation Accuracy:  0.17613095045089722
Iteration/Epoch:  20
Accuracy:  0.28353172540664673
Validation Accuracy:  0.28559523820877075
Iteration/Epoch:  30
Accuracy:  0.357103168964386
Validation Accuracy:  0.35035714507102966
Iteration/Epoch:  40
Accuracy:  0.42293649911880493
Validation Accuracy:  0.4151785671710968
Iteration/Epoch:  50
Accuracy:  0.47305554151535034
Validation Accuracy:  0.4645237922668457
Iteration/Epoch:  60
Accuracy:  0.5140476226806641
Validation Accuracy:  0.510357141494751
Iteration/Epoch:  70
Accuracy:  0.5515078902244568
Validation Accuracy:  0.5468452572822571
Iteration/Epoch:  80
Accuracy:  0.5897618532180786
Validation Accuracy:  0.5835714340209961
Iteration/Epoch:  90
Accuracy:  0.6231745481491089
Validation Accuracy:  0.6193452477455139
Iteration/Epoch:  100
Accuracy:  0.6522618532180786
Validation Accuracy:  

CUDA is WORKING NICELY NICEEEEEEEEEE